In [82]:
#imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [83]:
load_dotenv(override=True)
openai = OpenAI()

In [84]:
# for pushover
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user sound and starts with {pushover_user[0]}")
else:
    print("Pushover user not found")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("Pushover token not found")

Pushover user sound and starts with u
Pushover token found and starts with a


In [85]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token":pushover_token, "message":message}
    requests.post(pushover_url,payload)

In [86]:
push("Hey !")

Push: Hey !


In [87]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded":"ok"}

In [88]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded":"ok"}

In [89]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "the email address of this user"
            },
            "name":{
                "type": "string",
                "description": "the user's name, if they provided it"
            },
            "notes":{
                "type": "string",
                "description": "Any additional infomration about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [90]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [91]:
tools = [{"type": "function","function": record_unknown_question_json},
         {"type": "function","function": record_user_details_json}]

In [92]:
tools

[{'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['question'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Use this tool to record that a user is interested in being touch and provided an email address',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'the email address of this user'},
     'name': {'type': 'string',
      'description': "the user's name, if they provided it"},
     'notes': {'type': 'string',
      'description': "Any additional infomration about the conversation that's worth recording to give context"}},
    'required': ['email

In [93]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}, flush=True")

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result == record_unknown_question(**arguments)

        results.append({"role": "tool", "content": json.dumps(result),"tool_call_id":tool_call.id})
    return results

In [94]:
reader = PdfReader("me/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text
with open("me/summary.txt","r",encoding="utf-8") as f:
    summary = f.read()
name = "Avnish Kumar"

In [95]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [96]:
def chat(message, history):
    messages = [{"role":"system","content":system_prompt}] + history + [{"role":"user","content":message}]
    done = False
    while not done:
        response = openai.chat.completions.create(model="gpt-4o-mini",messages=messages,tools=tools)
        finish_reason = response.choices[0].finish_reason

        if finish_reason =="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content


In [97]:
chat = gr.ChatInterface(fn=chat)
print(chat)
chat.launch()

Gradio Blocks instance: 30 backend functions
--------------------------------------------
fn_index=0
 inputs:
 |-<gradio.components.textbox.Textbox object at 0x1181be810>
 outputs:
 |-<gradio.components.textbox.Textbox object at 0x1181be810>
 |-<gradio.components.state.State object at 0x117f0bf80>
fn_index=1
 inputs:
 |-<gradio.components.state.State object at 0x117f0bf80>
 |-<gradio.components.chatbot.Chatbot object at 0x117f75b20>
 outputs:
 |-<gradio.components.chatbot.Chatbot object at 0x117f75b20>
fn_index=2
 inputs:
 |-<gradio.components.state.State object at 0x117f0bf80>
 |-<gradio.components.state.State object at 0x1182ac3b0>
 outputs:
 |-<gradio.components.state.State object at 0x11827dd00>
 |-<gradio.components.chatbot.Chatbot object at 0x117f75b20>
fn_index=3
 inputs:
 |-<gradio.components.chatbot.Chatbot object at 0x117f75b20>
 outputs:
 |-<gradio.components.state.State object at 0x1182ac3b0>
 |-<gradio.components.state.State object at 0x118267020>
fn_index=4
 inputs:
 outp

Tool called: record_user_details, flush=True
Push: Recording interest from Name not provided with email avn.k@gmail.com and notes not provided
